In [1]:
import os
import glob
import tensorflow as tf
from datetime import datetime
from tb_cscs import tensorboard

In [2]:
image_shape = (224, 224)
batch_size = 128

def decode(serialized_example):
    features = tf.io.parse_single_example(
        serialized_example,
        features={
            'image/encoded': tf.io.FixedLenFeature([], tf.string),
            'image/class/label': tf.io.FixedLenFeature([], tf.int64),
        })
    image = tf.image.decode_jpeg(features['image/encoded'], channels=3)
    image = tf.image.resize(image, image_shape, method='bicubic')
    label = tf.cast(features['image/class/label'], tf.int64)
    return image, label-1

In [3]:
list_of_files = glob.glob('/scratch/snx3000/stud50/imagenet/*')

AUTO = tf.data.experimental.AUTOTUNE
dataset = (tf.data.TFRecordDataset(list_of_files, num_parallel_reads=AUTO)
           .map(decode, num_parallel_calls=AUTO)
           .batch(batch_size)
           .prefetch(AUTO)
          )

In [4]:
model = tf.keras.applications.InceptionV3(weights=None,
                                          input_shape=(*image_shape, 3),
                                          classes=1000)

optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=os.path.join('inceptionv3_logs',
                                                                  datetime.now().strftime("%d-%H%M")),
                                             histogram_freq=1,
                                             profile_batch='80,100')

In [6]:
fit = model.fit(dataset.take(100),
                epochs=1,
                callbacks=[tb_callback])

     99/Unknown - 45s 458ms/step - loss: 7.0105 - accuracy: 0.0013WARNING:tensorflow:From /apps/daint/UES/6.0.UP04/sandboxes/sarafael/miniconda-tf2.3/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
100/100 [==============================] - 52s 524ms/step - loss: 7.0102 - accuracy: 0.0013


In [7]:
%reload_ext tensorboard

In [8]:
%tensorboard --logdir=inceptionv3_logs

Reusing TensorBoard on port 6006 (pid 19557), started 6:07:35 ago. (Use '!kill 19557' to kill it.)